In [33]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3.1:latest", temperature=0)

In [34]:
from langgraph.graph import MessagesState

class State(MessagesState):
    summary:str
    


In [35]:
from langchain_core.messages import SystemMessage, HumanMessage, RemoveMessage


def call_model(state:State):
    summary  = state.get("summary", "")

    if(summary):
        sys_msg = f"summary of earlier conversation:{summary}"
        messages = [SystemMessage(content=sys_msg)+state["messages"]]

    else:
        messages =  state["messages"]
    
    response = llm.invoke(messages)
    return {"messages": response}

In [36]:
def summarize_message(state:State):
    summary  = state.get("summary", "")
    if(summary):
        summary_message = (
            f"This is summary of the conversation to date: {summary}\n\n"
            "Extend the summary by taking into account the new messages above:"
        )
    else:
       summary_message = "Create a summary of the conversation above:"

    messages = state["messages"]+[HumanMessage(content=summary_message)]
    response = llm.invoke(messages)

    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:-2]]
    return {"summary": response.content, "messages": delete_messages}



In [37]:
from langgraph.graph import END
# Determine whether to end or summarize the conversation
def should_continue(state: State):
    
    """Return the next node to execute."""
    
    messages = state["messages"]
    
    # If there are more than six messages, then we summarize the conversation
    if len(messages) > 6:
        return "summarize_message"
    
    # Otherwise we can just end
    return END

In [38]:
from langgraph.graph import START, END, StateGraph
from langgraph.checkpoint.memory import MemorySaver
workflow  = StateGraph(State)

workflow.add_node("conversation", call_model)
workflow.add_node("summarize_message", summarize_message)

workflow.add_edge(START, "conversation")
workflow.add_conditional_edges("conversation", should_continue)
workflow.add_edge("summarize_message", END)

memory = MemorySaver()
graph = workflow.compile(checkpointer=memory)


In [39]:
# Create a thread
config = {"configurable": {"thread_id": "1"}}

# Start conversation
input_message = HumanMessage(content="hi! I'm Lance")
output = graph.invoke({"messages": [input_message]}, config) 
for m in output['messages'][-1:]:
    m.pretty_print()

input_message = HumanMessage(content="what's my name?")
output = graph.invoke({"messages": [input_message]}, config) 
for m in output['messages'][-1:]:
    m.pretty_print()

input_message = HumanMessage(content="i like the 49ers!")
output = graph.invoke({"messages": [input_message]}, config) 
for m in output['messages'][-1:]:
    m.pretty_print()

================================== Ai Message ==================================

Hi Lance! What's up? How can I help you today?
================================== Ai Message ==================================

Your name is Lance! We just established that, didn't we?
================================== Ai Message ==================================

A Bay Area native, eh? The San Francisco 49ers are a legendary team with a rich history in the NFL. Who's your favorite player past or present?


In [40]:
graph.get_state(config).values.get("summary","")

''

In [41]:
input_message = HumanMessage(content="i like Nick Bosa, isn't he the highest paid defensive player?")
output = graph.invoke({"messages": [input_message]}, config) 
for m in output['messages'][-1:]:
    m.pretty_print()

================================== Ai Message ==================================

Nick Bosa is an incredible talent and a key contributor to the 49ers' defense! And yes, you're correct again - he signed a massive contract extension with the 49ers in 2021 that made him one of the highest-paid defensive players in the NFL. He's a dominant edge rusher who can get after the quarterback and make game-changing plays. The 49ers are lucky to have him on their roster!
